In [8]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df_warehouse = pd.read_parquet('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_4/df_all_store_VZ01.parquet')

In [17]:
folder_of_stock_sales = r'/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_4'
# read parquat df and concat  with os
df_stock_sales = pd.DataFrame()
for file in os.listdir(folder_of_stock_sales):
    df_stock_sales = pd.concat([df_stock_sales, pd.read_parquet(os.path.join(folder_of_stock_sales, file))])

,sku,date,stock
59974,100060075000001,2021-08-20,257.0
59975,100060075000001,2021-08-21,257.0
59976,100060075000001,2021-08-22,257.0
59977,100060075000001,2021-08-23,257.0
59978,100060075000001,2021-08-24,257.0
...,...,...,...
67753,100060075000001,2023-12-08,0.0
67754,100060075000001,2023-12-09,0.0
67755,100060075000001,2023-12-10,0.0
67756,100060075000001,2023-12-11,0.0


In [16]:
miss_sales = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_with_check_exp/MissedSales.csv')

,sku,store,date,average_price,average_dicounted_price,sales,item,stock
44011,100060075000001,100,2021-08-21,NaN,NaN,0.0,None,0.0
44012,100060075000001,100,2021-08-22,NaN,NaN,0.0,None,0.0
44013,100060075000001,100,2021-08-23,NaN,NaN,0.0,None,0.0
44014,100060075000001,100,2021-08-24,NaN,NaN,0.0,None,0.0
44015,100060075000001,100,2021-08-25,NaN,NaN,0.0,None,0.0
...,...,...,...,...,...,...,...,...
62358,100060075000001,100,2023-12-08,NaN,NaN,0.0,None,0.0
62359,100060075000001,100,2023-12-09,NaN,NaN,0.0,None,0.0
62360,100060075000001,100,2023-12-10,NaN,NaN,0.0,None,0.0
62361,100060075000001,100,2023-12-11,NaN,NaN,0.0,None,0.0


In [ ]:
miss_sales['date'] = pd.to_datetime(miss_sales['date'])

In [ ]:
miss_sales

In [ ]:
# take in amount_miss_sales_dict sum of amount_miss_sales_dict[sku, *]


In [ ]:
df_stock_sales['date'] = pd.to_datetime(df_stock_sales['date'])

In [ ]:
stocks = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_with_check_exp/stocks.csv')
stocks['date'] = pd.to_datetime(stocks['date'])

In [ ]:
print(stocks['sku'].unique())

In [ ]:
stocks

In [ ]:
df_warehouse

In [ ]:
df_warehouse[df_warehouse['stock'] > 0].groupby('sku')['date'].min()[int(sku)]

In [ ]:
print(df_stock_sales['sku'].unique())

In [ ]:
df_stock_sales['sku'] = df_stock_sales['sku'].astype(str)

In [ ]:
sku_stocks[(sku_stocks['stock']>0) & (sku_stocks['store'] == store)]

In [ ]:
for sku in ['100654238000002']:
    print(sku)
    min_date = df_warehouse[df_warehouse['stock'] > 0].groupby('sku')['date'].min()[int(sku)]
    sku_data = df_stock_sales[(df_stock_sales['sku'] == sku) ]
    sku_stocks = stocks[(stocks['sku'] == int(sku)) ]
    amount_miss_sales_dict = miss_sales[miss_sales['sku'] == int(sku)].groupby('store')['amount_miss_sales'].max().to_dict()
    for store in sku_data['store'].unique():

        if store == store:
            max_date = sku_data[(sku_data['sales']>0) & (sku_data['store'] == store)]['date'].iloc[-1]
            min_date = sku_data[(sku_data['sales']>0) & (sku_data['store'] == store)]['date'].iloc[0]
            stocks_store = sku_stocks[(sku_stocks['store'] == store) &  (sku_stocks['date'] >= min_date) & (sku_stocks['date'] <= max_date)].set_index('date')['stock']
            palmers_stock_store = sku_data[(sku_data['store'] == store) &  (sku_data['date'] >= min_date) & (sku_data['date'] <= max_date)].set_index('date')['stock']
            sales_store = sku_data[(sku_data['store'] == store) &  (sku_data['date'] >= min_date) & (sku_data['date'] <= max_date)].set_index('date')['sales']
            sum_of_our_stock = stocks_store.sum()
            sum_of_palmers_stock = palmers_stock_store.sum()
            our_saving_prec = np.round((sum_of_palmers_stock - sum_of_our_stock) * 100 / sum_of_palmers_stock, 2)
            palmers_stock_store_zero = palmers_stock_store[palmers_stock_store == 0]
            palmers_stock_store_zero = palmers_stock_store_zero.replace(0, max(palmers_stock_store))
            plt.figure(figsize=(20, 4))
            plt.plot(stocks_store, label='our stock')
            plt.plot(palmers_stock_store, label='palmers stock')
            plt.plot(sales_store, label='sales')
            plt.title(f'sku: {sku}, store: {store}\n'
                      f'sum_of_our_stock: {sum_of_our_stock}, sum_of_palmers_stock: {sum_of_palmers_stock}  our_saving_prec_in_stock: {our_saving_prec}%\n'
                      f'our miss sales: {amount_miss_sales_dict[int(store)]}, prec_of_time_of_palmers_stock_zero: {np.round(len(palmers_stock_store_zero) *100/len(palmers_stock_store), 2)}%')



            miss_sales[miss_sales['sku'] == int(sku)]
            miss_sales_store = amount_miss_sales_dict[int(store)]
            # plot y = miss_sales_store with dots on the graph with index of stocks_store
            plt.plot(pd.DataFrame([miss_sales_store]*len(stocks_store), index=stocks_store.index), linestyle='--', label='missed sales')
            # zero stock in palmers

            plt.scatter(palmers_stock_store_zero.index, palmers_stock_store_zero, label='palmers zero stock', color='red', marker='x', linestyle='')
            plt.legend()

            plt.show()

In [ ]:
sku

In [ ]:
stocks[stocks['sku'] == int(sku) ]

In [ ]:
stocks['sku'].unique()

In [ ]:
stocks['sku']

In [ ]:
sku_stocks

In [ ]:
sku_data[(sku_data['sales']>0) & (sku_stocks['store'] == store)]

In [ ]:
sku_data[sku_data['sales']>0]['date'].iloc[-1]

In [ ]:
max_date = sku_data[sku_data['sales']>0]['date'].max()

In [ ]:
max_date

In [ ]:
miss_sales[miss_sales['sku'] == int(sku)].groupby('store')['amount_miss_sales'].max().sum()


In [ ]:
miss_sales['sku'].unique()